Show Parquet / Pyarrow API.

## Imports

In [33]:
import logging
import os
import random

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import numpy as np

import helpers.dbg as dbg

dbg.init_logger(verbosity=logging.INFO)
_LOG = logging.getLogger(__name__)

In [6]:
def get_df() -> pd.DataFrame:
    """
    Create pandas random data, like:
    
                idx instr  val1  val2
    2000-01-01    0     A    99    30
    2000-01-02    0     A    54    46
    2000-01-03    0     A    85    86
    """
    num_rows = 100
    instruments = "A B C D E".split()
    cols = "id stock val1 val2".split()
    df_idx = pd.date_range(pd.Timestamp("2000-01-01"), pd.Timestamp("2000-01-15"), freq="1D")
    #print(df_idx)
    random.seed(1000)

    df = []
    for idx, inst in enumerate(instruments):
        df_tmp = pd.DataFrame({"idx": idx,
                               "instr": inst,
                               "val1": [random.randint(0, 100) for k in range(len(df_idx))],
                               "val2": [random.randint(0, 100) for k in range(len(df_idx))],
                              }, index=df_idx)
        #print(df_tmp)
        df.append(df_tmp)
    df = pd.concat(df)
    return df

            idx instr  val1  val2
2000-01-01    0     A    99    30
2000-01-02    0     A    54    46
2000-01-03    0     A    85    86
2000-01-04    0     A    97    62
2000-01-05    0     A    12    25


In [42]:
def df_to_str(df: pd.DataFrame) -> str:
    txt = ""
    txt += "# df=\n%s" % df.head(3)
    txt += "\n# df.shape=\n%s" % str(df.shape)
    txt += "\n# df.dtypes=\n%s" % str(df.dtypes)
    return txt

SyntaxError: EOL while scanning string literal (<ipython-input-42-7a84befc1958>, line 4)

# Save and load all data in one file

In [21]:
df = get_df()
#print(df.head())
print(df_to_str(df))

# df=
            idx instr  val1  val2
2000-01-01    0     A    99    30
2000-01-02    0     A    54    46
2000-01-03    0     A    85    86
# df.shape=
(75, 4)
# df.dtypes=
idx       int64
instr    object
val1      int64
val2      int64
dtype: object


In [22]:
table = pa.Table.from_pandas(df)

print("table=\n%s" % table)

table=
pyarrow.Table
idx: int64
instr: string
val1: int64
val2: int64
__index_level_0__: timestamp[ns]


In [23]:
# Save.
file_name = "df_in_one_file.pq"
pq.write_table(table, file_name)

In [25]:
# Load.
df2 = pq.read_table(file_name)
print(df2)

df2 = df2.to_pandas()
print(df_to_str(df2))

pyarrow.Table
idx: int64
instr: string
val1: int64
val2: int64
__index_level_0__: timestamp[us]
# df=
            idx instr  val1  val2
2000-01-01    0     A    99    30
2000-01-02    0     A    54    46
2000-01-03    0     A    85    86
# df.shape=
(75, 4)
# df.dtypes=
idx       int64
instr    object
val1      int64
val2      int64
dtype: object


## Read a subset of columns

In [27]:
df2 = pq.read_table(file_name, columns=["idx", "val1"])
print(df2)

df2 = df2.to_pandas()
print(df_to_str(df2))

pyarrow.Table
idx: int64
val1: int64
# df=
   idx  val1
0    0    99
1    0    54
2    0    85
# df.shape=
(75, 2)
# df.dtypes=
idx     int64
val1    int64
dtype: object


## Partitioned dataset

from https://arrow.apache.org/docs/python/dataset.html#reading-partitioned-data

- A dataset can exploit a nested structure, where the sub-dir names hold information about which subset of the data is stored in that dir
- E.g., "Hive" patitioning scheme "key=vale" dir names

In [29]:
df = get_df()
print(df_to_str(df))

# df=
            idx instr  val1  val2
2000-01-01    0     A    99    30
2000-01-02    0     A    54    46
2000-01-03    0     A    85    86
# df.shape=
(75, 4)
# df.dtypes=
idx       int64
instr    object
val1      int64
val2      int64
dtype: object


In [30]:
base = "."
dir_name =  os.path.join(base, "parquet_dataset_partitioned")
os.system("rm -rf %s" % dir_name)

pq.write_to_dataset(table,
                    dir_name,
                    partition_cols=['idx'])

In [31]:
!ls parquet_dataset_partitioned 

'idx=0'  'idx=1'  'idx=2'  'idx=3'  'idx=4'


In [34]:
# Read data back.
dataset = ds.dataset(dir_name,
                     format="parquet",
                     partitioning="hive")

print("\n".join(dataset.files))

./parquet_dataset_partitioned/idx=0/cab9de6eff0c47bcb688a1ce437c7f89.parquet
./parquet_dataset_partitioned/idx=1/56813e569097420cae892720d3bb0789.parquet
./parquet_dataset_partitioned/idx=2/5c9a17d2e1294dd58c7d8695868c2cb5.parquet
./parquet_dataset_partitioned/idx=3/b28576eb22d54999980a313a24511497.parquet
./parquet_dataset_partitioned/idx=4/8ee3f0d7585b48959a560c954562add8.parquet


In [37]:
# Read everything.
df2 = dataset.to_table().to_pandas()

print(df_to_str(df2))

# df=
           instr  val1  val2  idx
2000-01-01     A    99    30    0
2000-01-02     A    54    46    0
2000-01-03     A    85    86    0
# df.shape=
(75, 4)
# df.dtypes=
instr    object
val1      int64
val2      int64
idx       int32
dtype: object


In [40]:
# Load part of the data.

df2 = dataset.to_table(filter=ds.field("idx") == 1).to_pandas()
print(df_to_str(df2))

df2 = dataset.to_table(filter=ds.field("idx") < 3).to_pandas()
print(df_to_str(df2))

# df=
           instr  val1  val2  idx
2000-01-01     B    18    22    1
2000-01-02     B    59    89    1
2000-01-03     B    91    90    1
# df.shape=
(15, 4)
# df.dtypes=
instr    object
val1      int64
val2      int64
idx       int32
dtype: object
# df=
           instr  val1  val2  idx
2000-01-01     A    99    30    0
2000-01-02     A    54    46    0
2000-01-03     A    85    86    0
# df.shape=
(45, 4)
# df.dtypes=
instr    object
val1      int64
val2      int64
idx       int32
dtype: object


## Add year-month partitions

In [45]:
df = get_df()
df["year"] = df.index.year
df["month"] = df.index.month

print(df_to_str(df))

# df=
            idx instr  val1  val2  year  month
2000-01-01    0     A    99    30  2000      1
2000-01-02    0     A    54    46  2000      1
2000-01-03    0     A    85    86  2000      1
# df.shape=
(75, 6)
# df.dtypes=
idx       int64
instr    object
val1      int64
val2      int64
year      int64
month     int64
dtype: object


In [48]:
table = pa.Table.from_pandas(df)

print("table=\n%s" % table)

table=
pyarrow.Table
idx: int64
instr: string
val1: int64
val2: int64
year: int64
month: int64
__index_level_0__: timestamp[ns]


In [49]:
base = "."
dir_name =  os.path.join(base, "pq_partitioned2")
os.system("rm -rf %s" % dir_name)

pq.write_to_dataset(table,
                    dir_name,
                    partition_cols=['idx', "year", "month"])

In [51]:
!ls $dir_name

'idx=0'  'idx=1'  'idx=2'  'idx=3'  'idx=4'


In [50]:
!ls $dir_name/idx=0/year=2000/month=1

bc6b2314c7f640a38c62029280f6f65e.parquet


In [52]:
# Read data back.
dataset = ds.dataset(dir_name,
                     format="parquet",
                     partitioning="hive")

print("\n".join(dataset.files))

./pq_partitioned2/idx=0/year=2000/month=1/bc6b2314c7f640a38c62029280f6f65e.parquet
./pq_partitioned2/idx=1/year=2000/month=1/bb178ff0bdd344ca8328f9d67398b322.parquet
./pq_partitioned2/idx=2/year=2000/month=1/16081eea25fd4da6bd802037b541766c.parquet
./pq_partitioned2/idx=3/year=2000/month=1/1557b3c461054eadba16e3072fbd3a8a.parquet
./pq_partitioned2/idx=4/year=2000/month=1/07a0c7fcf054450296b35452b57236ef.parquet


In [56]:
# Read data back.
dataset = ds.dataset(dir_name,
                     format="parquet",
                     partitioning="hive")

df2 = dataset.to_table(filter=ds.field('idx') == 2).to_pandas()
print(df_to_str(df2))

# df=
           instr  val1  val2  idx  year  month
2000-01-01     C    99    37    2  2000      1
2000-01-02     C    98    48    2  2000      1
2000-01-03     C    70    58    2  2000      1
# df.shape=
(15, 6)
# df.dtypes=
instr    object
val1      int64
val2      int64
idx       int32
year      int32
month     int32
dtype: object


In [81]:
# We could scan manually and create the dirs manually if we don't want to add
# add a new dir.
base = "."
dir_name =  os.path.join(base, "parquet_dataset_partitioned2")
os.system("rm -rf %s" % dir_name)

grouped = df.groupby(lambda x: x.day)
for day, df_tmp in grouped:
    print(day, df_tmp)
    grouped2 = df_tmp.groupby("idx")
    for id_, df_tmp2 in grouped2:
        print(day, id_, df_tmp2)

1             idx instr  val1  val2
2000-01-01    0     A    99    30
2000-01-01    1     B    18    22
2000-01-01    2     C    99    37
2000-01-01    3     D     9    97
2000-01-01    4     E    41    38
1 0             idx instr  val1  val2
2000-01-01    0     A    99    30
1 1             idx instr  val1  val2
2000-01-01    1     B    18    22
1 2             idx instr  val1  val2
2000-01-01    2     C    99    37
1 3             idx instr  val1  val2
2000-01-01    3     D     9    97
1 4             idx instr  val1  val2
2000-01-01    4     E    41    38
2             idx instr  val1  val2
2000-01-02    0     A    54    46
2000-01-02    1     B    59    89
2000-01-02    2     C    98    48
2000-01-02    3     D    29    97
2000-01-02    4     E     0     6
2 0             idx instr  val1  val2
2000-01-02    0     A    54    46
2 1             idx instr  val1  val2
2000-01-02    1     B    59    89
2 2             idx instr  val1  val2
2000-01-02    2     C    98    48
2 3         

## Partition manually

In [118]:
from pyarrow.dataset import DirectoryPartitioning

partitioning = DirectoryPartitioning(pa.schema([("year", pa.int16()), ("month", pa.int8()), ("day", pa.int8())]))
print(partitioning.parse("/2009/11/3"))

#partitioning.discover()

(((year == 2009) and (month == 11)) and (day == 3))


ValueError: Neither field_names nor schema was passed; cannot infer field_names

## Read subset of columns for everything

In [33]:
import pyarrow.dataset as ds

import numpy as np

In [ ]:
# How to merge PQ files

# We can filter by year, month, stock and then all save in the same dir

In [85]:
partitioning = 

SyntaxError: invalid syntax (<ipython-input-85-fa88cdbe1cc4>, line 1)

In [87]:
df.schema

AttributeError: 'DataFrame' object has no attribute 'schema'